<a href="https://colab.research.google.com/github/shiranimehr/Capstone-Threee/blob/master/Copy_of_Capstone3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, we import required libraries.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import urllib
import os.path
from os import path
import matplotlib.image as mpimg 
import PIL
import random
import datetime

Mount drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Get meta data from URL:

In [ ]:
metaData = pd.read_csv('https://raw.githubusercontent.com/ieee8023/covid-chestxray-dataset/master/metadata.csv')
metaData.head()

,patientid,offset,sex,age,finding,RT_PCR_positive,survival,intubated,intubation_present,went_icu,in_icu,needed_supplemental_O2,extubated,temperature,pO2_saturation,leukocyte_count,neutrophil_count,lymphocyte_count,view,modality,date,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 29
0,2,0.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,N,Y,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
1,2,3.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,N,Y,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,"January 25, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
2,2,5.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,N,Y,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,"January 27, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
3,2,6.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,N,Y,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,"January 28, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
4,4,0.0,F,52.0,Pneumonia/Viral/COVID-19,Y,NaN,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,"January 25, 2020","Changhua Christian Hospital, Changhua City, Ta...",images,nejmc2001573_f1a.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,diffuse infiltrates in the bilateral lower lungs,NaN,NaN


In [ ]:
metaData.describe()


,offset,age,temperature,pO2_saturation,leukocyte_count,neutrophil_count,lymphocyte_count
count,697.000000,713.000000,78.000000,119.000000,16.000000,28.000000,40.000000
mean,9.083214,53.534362,38.898718,78.475630,5.024375,5.307143,4.638050
std,31.664077,16.949007,6.494383,38.742573,2.660341,3.489956,20.550033
min,-360.000000,18.000000,36.000000,20.000000,0.220000,0.000000,0.400000
25%,1.000000,40.000000,37.800000,55.000000,3.067500,2.725000,0.746500
50%,5.000000,54.000000,38.150000,85.000000,4.745000,4.800000,1.100000
75%,10.000000,68.000000,38.975000,95.000000,6.857500,6.925000,1.600000
max,365.000000,94.000000,95.000000,340.000000,11.200000,12.900000,131.000000


In [ ]:
metaData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950 entries, 0 to 949
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   patientid               950 non-null    object 
 1   offset                  697 non-null    float64
 2   sex                     870 non-null    object 
 3   age                     713 non-null    float64
 4   finding                 950 non-null    object 
 5   RT_PCR_positive         593 non-null    object 
 6   survival                361 non-null    object 
 7   intubated               248 non-null    object 
 8   intubation_present      250 non-null    object 
 9   went_icu                397 non-null    object 
 10  in_icu                  335 non-null    object 
 11  needed_supplemental_O2  90 non-null     object 
 12  extubated               37 non-null     object 
 13  temperature             78 non-null     float64
 14  pO2_saturation          119 non-null    fl

In [ ]:
metaData['date'].fillna(method='ffill', inplace=True)

Drop extra columns:

In [ ]:
drop_column_list = ['needed_supplemental_O2', 'extubated', 
                    'temperature', 'pO2_saturation', 'leukocyte_count',
                    'neutrophil_count', 'lymphocyte_count',
                    'doi','url', 'license', 'clinical_notes', 'other_notes']
metaData.drop(drop_column_list, axis=1, inplace=True)

Fill NAs:

In [ ]:
metaData['sex'].fillna('Unknown', inplace=True)
metaData['RT_PCR_positive'].fillna('Unknown', inplace=True)
metaData['survival'].fillna('Unknown', inplace=True)
metaData['intubated'].fillna('Unknown', inplace=True)
metaData['intubation_present'].fillna('Unknown', inplace=True)
metaData['went_icu'].fillna('Unknown', inplace=True)
metaData['in_icu'].fillna('Unknown', inplace=True)
metaData['location'].fillna('Unknown', inplace=True)

In [ ]:
metaData['offset'].fillna(np.mean(metaData['offset']), inplace=True)
metaData['age'].fillna(np.mean(metaData['age']), inplace=True)

In [ ]:
 metaData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950 entries, 0 to 949
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   patientid           950 non-null    object 
 1   offset              950 non-null    float64
 2   sex                 950 non-null    object 
 3   age                 950 non-null    float64
 4   finding             950 non-null    object 
 5   RT_PCR_positive     950 non-null    object 
 6   survival            950 non-null    object 
 7   intubated           950 non-null    object 
 8   intubation_present  950 non-null    object 
 9   went_icu            950 non-null    object 
 10  in_icu              950 non-null    object 
 11  view                950 non-null    object 
 12  modality            950 non-null    object 
 13  date                950 non-null    object 
 14  location            950 non-null    object 
 15  folder              950 non-null    object 
 16  filename

Check for duplicates:

In [ ]:
metaData.drop_duplicates(inplace=True)

In [ ]:
metaData['finding'].unique()

array(['Pneumonia/Viral/COVID-19', 'Pneumonia', 'Pneumonia/Viral/SARS',
       'Pneumonia/Fungal/Pneumocystis',
       'Pneumonia/Bacterial/Streptococcus', 'No Finding',
       'Pneumonia/Bacterial/Chlamydophila', 'Pneumonia/Bacterial/E.Coli',
       'Pneumonia/Bacterial/Klebsiella', 'Pneumonia/Bacterial/Legionella',
       'Unknown', 'Pneumonia/Lipoid', 'Pneumonia/Viral/Varicella',
       'Pneumonia/Bacterial', 'Pneumonia/Bacterial/Mycoplasma',
       'Pneumonia/Viral/Influenza', 'todo', 'Tuberculosis',
       'Pneumonia/Viral/Influenza/H1N1', 'Pneumonia/Fungal/Aspergillosis',
       'Pneumonia/Viral/Herpes ', 'Pneumonia/Aspiration',
       'Pneumonia/Bacterial/Nocardia', 'Pneumonia/Viral/MERS-CoV',
       'Pneumonia/Bacterial/Staphylococcus/MRSA'], dtype=object)

Add a binary column for COVID-19: 

In [ ]:
metaData['COVID-19'] = metaData['finding'].apply(lambda x: True if ('COVID-19' in x) else False)

Get images and drop records with no image:

In [ ]:
#image_directory = '../Data/images'
url = "https://raw.githubusercontent.com/ieee8023/covid-chestxray-dataset/master/images/"

remove_row_list = []
for i in range(metaData.shape[0]):
    filename = metaData.iloc[i,:]['filename']
    try:
      ret = urllib.request.urlopen(url+filename)
    except:
      print(i, filename, "not found")
      remove_row_list.append(i)

metaData.drop(remove_row_list, axis=0, inplace=True)

53 radiopaedia_org_covid-19-pneumonia-7_85703_0-dcm.nii.gz not found
56 radiopaedia_org_covid-19-pneumonia-14_85914_0-dcm.nii.gz not found
225 coronacases_org_001.nii.gz not found
226 coronacases_org_002.nii.gz not found
227 coronacases_org_003.nii.gz not found
228 coronacases_org_004.nii.gz not found
229 coronacases_org_005.nii.gz not found
230 coronacases_org_006.nii.gz not found
231 coronacases_org_007.nii.gz not found
232 coronacases_org_008.nii.gz not found
233 coronacases_org_009.nii.gz not found
234 coronacases_org_010.nii.gz not found
235 radiopaedia_org_covid-19-pneumonia-4_85506_1-dcm.nii.gz not found
236 radiopaedia_org_covid-19-pneumonia-29_86490_1-dcm.nii.gz not found
237 radiopaedia_org_covid-19-pneumonia-29_86491_1-dcm.nii.gz not found
241 radiopaedia_org_covid-19-pneumonia-23_86359_0-dcm.nii.gz not found
242 radiopaedia_org_covid-19-pneumonia-10_85902_1-dcm.nii.gz not found
243 radiopaedia_org_covid-19-pneumonia-10_85902_3-dcm.nii.gz not found
244 radiopaedia_org_covid-

In [ ]:
metaData.shape

(929, 19)

In [ ]:
metaData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 929 entries, 0 to 949
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   patientid           929 non-null    object 
 1   offset              929 non-null    float64
 2   sex                 929 non-null    object 
 3   age                 929 non-null    float64
 4   finding             929 non-null    object 
 5   RT_PCR_positive     929 non-null    object 
 6   survival            929 non-null    object 
 7   intubated           929 non-null    object 
 8   intubation_present  929 non-null    object 
 9   went_icu            929 non-null    object 
 10  in_icu              929 non-null    object 
 11  view                929 non-null    object 
 12  modality            929 non-null    object 
 13  date                929 non-null    object 
 14  location            929 non-null    object 
 15  folder              929 non-null    object 
 16  filename

Copying images to two folders: Positive and Negative:

In [ ]:
#import requests
covid_positive = metaData[metaData['COVID-19']]
#for i in range(len(covid_positive)):
#    print(i+1, 'of', len(covid_positive))
#    filename = covid_positive.iloc[i,:]['filename']
#    r = requests.get(url+filename, stream = True)  
#    with open("/content/drive/My Drive/Capstone3-Images/Positive/"+filename, "wb") as file:  
#      for block in r.iter_content(chunk_size = 1024): 
#         if block:  
#             file.write(block)
print('Number of positive cases:', len(covid_positive) )

covid_negative = metaData[metaData['COVID-19']==False]
#for i in range(len(covid_negative)):
#    filename = covid_negative.iloc[i,:]['filename']
#    print(i+1, 'of', len(covid_negative))
#    r = requests.get(url+filename, stream = True)  
#    with open("/content/drive/My Drive/Capstone3-Images/Negative/"+filename, "wb") as file:  
#      for block in r.iter_content(chunk_size = 1024): 
#         if block:  
#             file.write(block)
print('Number of negative cases:', len(covid_negative) )

Number of positive cases: 563
Number of negative cases: 366


Next, let's change all image sizes to 256x256.

In [ ]:
#import cv2
#for i in range(len(covid_positive)):
#    filename = covid_positive.iloc[i,:]['filename']
#    im = cv2.imread("/content/drive/My Drive/Capstone3-Images/Positive/"+filename)
#    resized = cv2.resize(im, (256,256))
#    cv2.imwrite("/content/drive/My Drive/Capstone3-Images-Resized/Positive/"+filename, resized)

#for i in range(len(covid_negative)):
#    filename = covid_negative.iloc[i,:]['filename']
#    im = cv2.imread("/content/drive/My Drive/Capstone3-Images/Negative/"+filename)
#    resized = cv2.resize(im, (256,256))
#    cv2.imwrite("/content/drive/My Drive/Capstone3-Images-Resized/Negative/"+filename, resized)

Data augmentation:

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

for i in range(len(covid_positive)):
    print(i,' of ', len(covid_positive))
    filename = covid_positive.iloc[i,:]['filename']
    img = load_img("/content/drive/My Drive/Capstone3-Images-Resized/Positive/"+filename) 
    x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

    # the .flow() command below generates batches of randomly transformed images
    # and saves the results to the `preview/` directory
    i = 0
    for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='/content/drive/My Drive/Capstone3-Images-Resized-Augmented/Positive', save_prefix=filename.split('.')[0], save_format='jpeg'):
      i += 1
      if i > 10:
          break  # otherwise the generator would loop indefinitely


0  of  563
1  of  563
2  of  563
3  of  563
4  of  563
5  of  563
6  of  563
7  of  563
8  of  563
9  of  563
10  of  563
11  of  563
12  of  563
13  of  563
14  of  563
15  of  563
16  of  563
17  of  563
18  of  563
19  of  563
20  of  563
21  of  563
22  of  563
23  of  563
24  of  563
25  of  563
26  of  563
27  of  563
28  of  563
29  of  563
30  of  563
31  of  563
32  of  563
33  of  563
34  of  563
35  of  563
36  of  563
37  of  563
38  of  563
39  of  563
40  of  563
41  of  563
42  of  563
43  of  563
44  of  563
45  of  563
46  of  563
47  of  563
48  of  563
49  of  563
50  of  563
51  of  563
52  of  563
53  of  563
54  of  563
55  of  563
56  of  563
57  of  563
58  of  563
59  of  563
60  of  563
61  of  563
62  of  563
63  of  563
64  of  563
65  of  563
66  of  563
67  of  563
68  of  563
69  of  563
70  of  563
71  of  563
72  of  563
73  of  563
74  of  563
75  of  563
76  of  563
77  of  563
78  of  563
79  of  563
80  of  563
81  of  563
82  of  563
83  of  563
84

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

for i in range(len(covid_negative)):
    print(i+1,' of ', len(covid_negative))
    filename = covid_negative.iloc[i,:]['filename']
    img = load_img("/content/drive/My Drive/Capstone3-Images-Resized/Negative/"+filename) 
    x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

    # the .flow() command below generates batches of randomly transformed images
    # and saves the results to the `preview/` directory
    i = 0
    for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='/content/drive/My Drive/Capstone3-Images-Resized-Augmented/Negative', save_prefix=filename.split('.')[0], save_format='jpeg'):
      i += 1
      if i > 10:
          break  # otherwise the generator would loop indefinitely



0  of  366
ARDSSevere.png
1  of  366
SARS-10.1148rg.242035193-g04mr34g0-Fig8a-day0.jpeg
2  of  366
SARS-10.1148rg.242035193-g04mr34g0-Fig8b-day5.jpeg
3  of  366
SARS-10.1148rg.242035193-g04mr34g0-Fig8c-day10.jpeg
4  of  366
SARS-10.1148rg.242035193-g04mr34g04a-Fig4a-day7.jpeg
5  of  366
SARS-10.1148rg.242035193-g04mr34g04b-Fig4b-day12.jpeg
6  of  366
SARS-10.1148rg.242035193-g04mr34g05x-Fig5-day9.jpeg
7  of  366
SARS-10.1148rg.242035193-g04mr34g07a-Fig7a-day5.jpeg
8  of  366
SARS-10.1148rg.242035193-g04mr34g07b-Fig7b-day12.jpeg
9  of  366
SARS-10.1148rg.242035193-g04mr34g09a-Fig9a-day17.jpeg
10  of  366
SARS-10.1148rg.242035193-g04mr34g09b-Fig9b-day19.jpeg
11  of  366
SARS-10.1148rg.242035193-g04mr34g09c-Fig9c-day27.jpeg
12  of  366
acute-respiratory-distress-syndrome-ards-1.jpg
13  of  366
acute-respiratory-distress-syndrome-ards.jpg
14  of  366
ards-secondary-to-tiger-snake-bite.png
15  of  366
pneumocystis-pneumonia-2-PA.png
16  of  366
pneumocystis-pneumonia-2-L.png
17  of  366
str

Import required libraries:

In [ ]:
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

Define base model and freeze it:

In [ ]:
base_model = MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

# Freeze the base_model
base_model.trainable = False

Create new model on top of the base model:

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
#x = Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
#x = Dense(1024,activation='relu')(x) #dense layer 2
#x = Dense(512,activation='relu')(x) #dense layer 3
x = keras.layers.Dropout(0.5)(x)
preds = Dense(2,activation='softmax')(x) #final layer with softmax activation

model = Model(inputs=base_model.input,outputs=preds)

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)   

Check number of layers:

In [ ]:
print(len(base_model.layers))
print(len(model.layers))

87
90


Split images into train and validation:

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2) #included in our dependencies

train_generator = train_datagen.flow_from_directory("/content/drive/My Drive/Capstone3-Images-Resized/", # this is where you specify the path to the main data folder
                                                 target_size=(256,256),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True,
                                                 subset='training')

validation_generator = train_datagen.flow_from_directory("/content/drive/My Drive/Capstone3-Images-Resized/", # this is where you specify the path to the main data folder
                                                 target_size=(256,256),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True,
                                                 subset='validation')


Found 744 images belonging to 2 classes.
Found 185 images belonging to 2 classes.


Train the top layer:

In [ ]:
#model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

#model.fit_generator(generator = train_generator,
#                    steps_per_epoch = train_generator.samples // train_generator.batch_size,
#                    validation_data = validation_generator, 
#                    validation_steps = validation_generator.samples // validation_generator.batch_size,
#                    epochs = 20)


In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

epochs = 20

model.fit_generator(generator = train_generator,
                    steps_per_epoch = train_generator.samples // train_generator.batch_size,
                    validation_data = validation_generator, 
                    validation_steps = validation_generator.samples // validation_generator.batch_size,
                    epochs = epochs)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
23/23 [==============================] - 43s 2s/step - loss: 1.0399 - accuracy: 0.5618 - val_loss: 0.6247 - val_accuracy: 0.6562
Epoch 2/20
23/23 [==============================] - 44s 2s/step - loss: 0.9185 - accuracy: 0.5983 - val_loss: 0.6202 - val_accuracy: 0.6875
Epoch 3/20
23/23 [==============================] - 43s 2s/step - loss: 0.8244 - accuracy: 0.6264 - val_loss: 0.5799 - val_accuracy: 0.6687
Epoch 4/20
23/23 [==============================] - 43s 2s/step - loss: 0.7713 - accuracy: 0.6306 - val_loss: 0.5593 - val_accuracy: 0.6625
Epoch 5/20
23/23 [==============================] - 43s 2s/step - loss: 0.6925 - accuracy: 0.6798 - val_loss: 0.5853 - val_accuracy: 0.6750
Epoch 6/20
23/23 [==============================] - 43s 2s/step - loss: 0.6181 - accuracy: 0.6966 - val_loss: 0.5481 - val_accuracy: 0.7250
Epoch 7/20
23/23 [==============================] - 45s 2s/step - loss: 0.6219 - acc

Finally, let's unfreeze the base model and train the entire model end-to-end with a low learning rate.

In [ ]:
base_model.trainable = True
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)   

In [ ]:

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

epochs = 20

model.fit_generator(generator = train_generator,
                    steps_per_epoch = train_generator.samples // train_generator.batch_size,
                    validation_data = validation_generator, 
                    validation_steps = validation_generator.samples // validation_generator.batch_size,
                    epochs = epochs)

Epoch 1/20
23/23 [==============================] - 185s 8s/step - loss: 1.3579 - accuracy: 0.6250 - val_loss: 0.5388 - val_accuracy: 0.7312
Epoch 2/20
23/23 [==============================] - 175s 8s/step - loss: 1.2162 - accuracy: 0.6503 - val_loss: 0.5362 - val_accuracy: 0.7563
Epoch 3/20
23/23 [==============================] - 180s 8s/step - loss: 0.9945 - accuracy: 0.6868 - val_loss: 0.5384 - val_accuracy: 0.7437
Epoch 4/20
23/23 [==============================] - 177s 8s/step - loss: 0.8478 - accuracy: 0.6994 - val_loss: 0.5487 - val_accuracy: 0.7437
Epoch 5/20
23/23 [==============================] - 178s 8s/step - loss: 0.7525 - accuracy: 0.7233 - val_loss: 0.5563 - val_accuracy: 0.7500
Epoch 6/20
 2/23 [=>............................] - ETA: 1:22 - loss: 0.5036 - accuracy: 0.7656

KeyboardInterrupt: ignored